<img src="https://i.ibb.co/qjt4Ymb/2022-09-19-004719.png" alt="2022-09-19-004719" border="0">

# 亞洲大學112深度學習教材(AUDL112-W4-Notebook1)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

# PyTorch 基礎教學

##  載入PyTorch 所需的函數庫


In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
print(torch.__version__)

1.12.1+cu113


# Part-1 The training phase

## IPO-M: Building a DL model
$M(x)=\hat{y}$

In [ ]:
#Building a DL model
device = "cuda"
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## IPO-I: input data

In [ ]:
#input data
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## IPO-O: ontput data

In [ ]:
#ontput data

## IPO-P: Training model

###  選擇一個優化器和損失函數進行模型建構


In [ ]:
#Create a loss function.
loss_fn = nn.CrossEntropyLoss()

In [ ]:
#Create an optimizer with the desired parameters.
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

### Compile model

In [ ]:
#model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])

##  進行模型訓練

In [ ]:
#model.fit(x_train, y_train, epochs=10)
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
loss: 2.181355  [    0/60000]
loss: 2.166751  [ 6400/60000]
loss: 2.110248  [12800/60000]
loss: 2.122288  [19200/60000]
loss: 2.069481  [25600/60000]
loss: 2.014033  [32000/60000]
loss: 2.041194  [38400/60000]
loss: 1.961552  [44800/60000]
loss: 1.982170  [51200/60000]
loss: 1.877516  [57600/60000]
Epoch 2
-------------------------------
loss: 1.940587  [    0/60000]
loss: 1.905509  [ 6400/60000]
loss: 1.793527  [12800/60000]
loss: 1.822840  [19200/60000]
loss: 1.712317  [25600/60000]
loss: 1.664944  [32000/60000]
loss: 1.678439  [38400/60000]
loss: 1.581435  [44800/60000]
loss: 1.618146  [51200/60000]
loss: 1.478765  [57600/60000]
Epoch 3
-------------------------------
loss: 1.596906  [    0/60000]
loss: 1.558632  [ 6400/60000]
loss: 1.415183  [12800/60000]
loss: 1.471257  [19200/60000]
loss: 1.354327  [25600/60000]
loss: 1.350772  [32000/60000]
loss: 1.357858  [38400/60000]
loss: 1.281927  [44800/60000]
loss: 1.323277  [51200/60000]
loss: 1.20

## IPO-P: Evaluating model (模型評估)

In [ ]:
#model.evaluate(x_test,  y_test)
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# Part-2 The Inference phase

## IPO-M: Predicting the digit of an image
$M(x)=\hat{y}$

In [ ]:
import numpy as np
from PIL import Image
from IPython.display import display

## IPO-I2: input a single image

In [ ]:
### 第一次

In [ ]:
#show image real size
img = Image.open('n1.bmp')
display(img)
img2=img.resize((28, 28))
img.close()

In [ ]:
#show image at the size of 28x28
display(img2)

### Covert an image to a numpy.ndarray (1, 28, 28)

In [ ]:
im2arr = np.array(img2)/ 255.0
im2arr = im2arr.reshape(1,28,28)

## IPO-P2: model.predict

In [ ]:
y_pred = model.predict(im2arr)
print(y_pred)


In [ ]:
y_hat =np.argmax(y_pred)
y_hat